In [31]:
!pip install -qU langchain "unstructured[all-docs]" pydantic lxml langchain-qdrant langchain_community pymupdf langchain-qdrant langchain-ollama

In [ ]:
import os
import importlib
import data_preprocessing
importlib.reload(data_preprocessing)
from data_preprocessing import data_preprocessing
from langchain_ollama import OllamaEmbeddings
from vector_store import vector_store
from uuid import uuid4

In [64]:
def file_names(directory):
    files=[]
    for file in os.listdir(directory):
        if file.endswith(".pdf"):
            files.append(os.path.join(directory, file))
    return files

In [65]:
directory = "./"

file_path = file_names(directory)
print(file_path)
doc=[]
for f in file_path:
    dp = data_preprocessing(file_path=f)
    doc += dp.load_data()

['./Brief Digital-Reward-Solution-for-JIBY (1).pdf', './RFP Classic template_from GIP Sharepoint.pdf']


In [62]:
embeddings = OllamaEmbeddings(model="llama3.2:latest")
vs = vector_store(collection_name="jti_rfp",embeddings=embeddings).vector_qdrant()

In [66]:
uuids = [str(uuid4()) for _ in range(len(doc))]
vs.add_documents(documents=doc, ids=uuids)

['dc86113d-4bec-40b2-9de2-922143c636ec',
 'b31cb3f7-12a9-4e89-9064-34c06454f574',
 '71ecebda-c04c-4599-abfd-cb9bf882659e',
 'c5572d53-bf30-4522-ae80-f38caa9a5271',
 'b016d871-0200-4681-95a8-a8403cd0520b',
 '7bd089eb-449c-4cc7-b54d-2355efe8a507',
 '069fd9c8-674e-4cb8-adb7-7efda4790a4b',
 '5830c052-3b22-4e0a-ba2f-81bdf41e9402',
 '37f620fd-a5c4-4274-9237-b8455397ee77',
 '76f82a3a-9c62-4016-a945-561fac2bd923',
 '8c36f597-9225-4b82-840a-171c08d08004',
 '54806c08-76a6-42f9-a267-25a3adf15e73',
 'ea84004c-e7cf-4e2c-a483-cab787800cd6',
 'a25a0fad-23d2-47c0-8b64-7650b55a21a8',
 'c20b326d-c900-483c-bc90-2cfd87a73d64',
 '0c04515f-9d7e-4a2a-bcd9-2fbf39a60efc',
 '59614e5f-2747-40e9-8dee-e30a93bebec9',
 '6bdb132c-2026-4043-a886-bb4abfe610f0',
 '50a52feb-0c60-4f2b-ba55-9ad82fe1a9aa',
 '13c09620-c513-4c09-8fc3-921075d3ebf6',
 '2a8bf24b-5ac6-4a78-960e-c7a09141ff67',
 'a7a1ef9d-3fe3-45c8-ac43-fc9d072c6b6e']